In [14]:
import functools
import os
import shlex
import subprocess

In [2]:
from fabric import api as fapi
from fabric import network as fnet
import paramiko

In [3]:
fapi.env.warn_only = True
fapi.env.use_ssh_config = True
user = fapi.env.user = 'cc'
host = fapi.env.host_string = '129.114.108.248'

In [4]:
def run(command, **kwargs):
    runargs = {
        'stdout': subprocess.PIPE,
        'stderr': subprocess.PIPE,
        'universal_newlines': True,
        'shell': False
    }
    runargs.update(kwargs)
    if not runargs['shell']:
        command = shlex.split(command)
    return subprocess.run(command, **runargs)

In [5]:
# proc = subprocess.run('git -C CC-Ubuntu16.04 status', shell=True, 
#           stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
# print(proc.stdout)

In [84]:
import errno
import time

from paramiko import SSHException
from fabric.network import NetworkError

In [134]:
# import io
# buf = io.BytesIO()
# fapi.get('/etc/os-release', buf)
# print(buf.getvalue().decode('ascii'))
while True:
    try:
        print(fapi.run('cat /etc/os-release', quiet=True))
        break
    except OSError as e:
        if e.errno == errno.ENETUNREACH:
            pass
        raise
    except (NetworkError, SSHException):
        pass
    print('.', end='')
    time.sleep(10)

NAME="Ubuntu"
VERSION="16.04.1 LTS (Xenial Xerus)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 16.04.1 LTS"
VERSION_ID="16.04"
HOME_URL="http://www.ubuntu.com/"
SUPPORT_URL="http://help.ubuntu.com/"
BUG_REPORT_URL="http://bugs.launchpad.net/ubuntu/"
VERSION_CODENAME=xenial
UBUNTU_CODENAME=xenial


In [135]:
print(fapi.run('uname -r', quiet=True))

4.4.0-57-generic


In [136]:
proc = run('git status', cwd='CC-Ubuntu16.04')
print(proc.stdout)
head = run('git rev-parse HEAD', cwd='CC-Ubuntu16.04').stdout.strip()
head

On branch gpu
nothing to commit, working tree clean



'299a3ad78331c735014de40baed03cb2c43112cb'

In [137]:
fapi.run('rm -rf ~/build.git', quiet=True)
out = fapi.run('git init --bare build.git', quiet=True)
print(out)

Initialized empty Git repository in /home/cc/build.git/


In [138]:
import os
os.getcwd()

'/Users/npt/Code/Ar/dibs'

In [139]:
proc = run('git push --all ssh://{}@{}/~/build.git'.format(user, host), cwd='CC-Ubuntu16.04', env={
    'GIT_SSH_COMMAND': 'ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no',
})
print(proc.returncode)
print(proc.stdout)
print(proc.stderr)

0

To ssh://129.114.108.248/~/build.git
 * [new branch]      cuda-submission -> cuda-submission
 * [new branch]      gpu -> gpu
 * [new branch]      master -> master



In [140]:
fapi.run('rm -rf ~/build', quiet=True)
fapi.run('git clone ~/build.git ~/build', quiet=True)
fapi.run('git -C ~/build -c advice.detachedHead=false checkout {}'.format(head))
print(fapi.run('ls -a build', quiet=True))

[129.114.108.248] run: git -C ~/build -c advice.detachedHead=false checkout 299a3ad78331c735014de40baed03cb2c43112cb
[129.114.108.248] out: HEAD is now at 299a3ad... Drop 'image type' as it's fixed
[129.114.108.248] out: 

.   create-image.sh  .git	 install-reqs.sh  VERSION
..  elements	     .gitignore  README.md


In [130]:
# out = fapi.run('sudo bash ~/build/install-reqs.sh', pty=True, quiet=True)
out = fapi.sudo('apt-get -qq install -y qemu-utils python-pip', quiet=True)
out = fapi.sudo('pip install diskimage-builder', quiet=True)
print(out[-300:])

r)
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [131]:
import io
out = io.StringIO()
# err = io.StringIO()
with fapi.cd('/home/cc/build/'):
#     out = fapi.run('bash create-image.sh', pty=False, quiet=True)
    fapi.run('bash create-image.sh', pty=True, quiet=False, capture_buffer_size=5000, stdout=out)#, stderr=err)
    
with open('build.log', 'w') as f:
    print(f.write(out.getvalue()))

[129.114.108.248] run: bash create-image.sh

1013882


In [109]:
print(out.getvalue()[-2000:])
print(len(out.getvalue()))

14.108.248] out: sudo: unable to resolve host image-builder-gpu
[129.114.108.248] out: Converting image using qemu-img convert
[129.114.108.248] out: Image file /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2 created...
[129.114.108.248] out: sudo: unable to resolve host image-builder-gpu
[129.114.108.248] out: + '[' -f /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2.qcow2 ']'
[129.114.108.248] out: + COMPRESSED_OUTPUT_FILE=/tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2-compressed
[129.114.108.248] out: + qemu-img convert /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2 -O qcow2 -c /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2-compressed
[129.114.108.248] out: + echo 'mv /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2-compressed /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2'
[129.114.108.248] out: mv /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2-compressed /tmp/tmp.7g2B322gMa/common/CC-Ubuntu16.04-CUDA8.qcow2
[129.114.108.248] out: 

In [132]:
out.seek(0)
ibi = f'[{host}] out: Image built in '
for line in out:
    if not line.startswith(ibi):
        continue
    output_file = line[len(ibi):].strip()
output_file

'/tmp/tmp.3VW9wNQiG0/common/CC-Ubuntu16.04.qcow2'

In [133]:
checksum = fapi.run(f'md5sum {output_file}', quiet=True).split()[0]
checksum

'c6c933d5c6d4e25c5399e5cb47327838'

uploaded my RC and ran this command with an SSH shell...

```bash
glance image-create --name "test-CC-Ubuntu16.04-CUDA8" --disk-format qcow2 --container-format bare --file gpu
```

rebuilt the instance with that image

In [42]:
fapi.run('ls', quiet=True)

"**************************************************************************\r\n# This system is using the EC2 Metadata Service, but does not appear to  #\r\n# be running on Amazon EC2 or one of cloud-init's known platforms that   #\r\n# provide a EC2 Metadata service. In the future, cloud-init may stop     #\r\n# reading metadata from the EC2 Metadata Service unless the platform can #\r\n# be identified.                                                         #\r\n#                                                                        #\r\n# If you are seeing this message, please file a bug against              #\r\n# cloud-init at                                                          #\r\n#    https://bugs.launchpad.net/cloud-init/+filebug?field.tags=dsid      #\r\n# Make sure to include the cloud provider your instance is               #\r\n# running on.                                                            #\r\n#                                                              

In [63]:
fapi.run('rm -rf ubu ubu.git', quiet=True)

''

In [28]:
options = {
    'username': 'cc',
    'hostname': '129.114.109.148',
}

client = paramiko.SSHClient()
client._policy = paramiko.WarningPolicy()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh_config = paramiko.SSHConfig()
user_config_file = os.path.expanduser("~/.ssh/config")
if os.path.exists(user_config_file):
    with open(user_config_file) as f:
        ssh_config.parse(f)

cfg = {'hostname': options['hostname'], 'username': options["username"]}

user_config = ssh_config.lookup(cfg['hostname'])
for k in ('hostname', 'username', 'port'):
    if k in user_config:
        cfg[k] = user_config[k]

if 'proxycommand' in user_config:
    cfg['sock'] = paramiko.ProxyCommand(user_config['proxycommand'])

client.connect(**cfg)

In [32]:
stdin, stdout, stderr = client.exec_command('ls')

In [36]:
stdout.read()

b''